In [ ]:
from data_retrieval.google_analytics_data import GoogleAnalyticsData
from datetime import date
CLIENT_SECRETS_FILE_PATH = './credentials/credentials.json'
TOKEN_FILE_PATH = './credentials/token.json'
ga_data = GoogleAnalyticsData(CLIENT_SECRETS_FILE_PATH, TOKEN_FILE_PATH)
run_id = ga_data.save_data(date(2022,1,1), date(2023,1,1))

In [ ]:
print('run id ', run_id)

In [ ]:
from helpers.cu_dataset_reader import read_cu_dataset_settings_pdf
pdf_file = './settings/CU datasets settings _ SAcommunity - Connecting Up Australia.pdf'
datasets_df = read_cu_dataset_settings_pdf(pdf_file, return_dataframe=True)


In [ ]:
datasets_df